# 📊 Comparaison des Modèles de Prédiction
## Hôpital Pitié-Salpêtrière

Ce notebook compare les performances de différents modèles de prédiction d'admissions hospitalières.

### Modèles testés :
1. **ARIMA** - Modèle statistique classique
2. **Prophet** - Modèle Facebook pour séries temporelles
3. **Random Forest** - Ensemble d'arbres de décision
4. **Gradient Boosting** - Arbres séquentiels
5. **Ensemble** - Combinaison pondérée

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from datetime import datetime, timedelta

# Style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("📦 Bibliothèques chargées")

## 1. Chargement des Données

In [ ]:
# Charger les données d'établissement
df = pd.read_csv('../data/etablissement.csv')
df['date'] = pd.to_datetime(df['date'])

print(f"📅 Période : {df['date'].min().date()} → {df['date'].max().date()}")
print(f"📊 Nombre de jours : {len(df)}")
print(f"🏥 Admissions moyennes : {df['nb_admissions'].mean():.0f}/jour")
print(f"   Min : {df['nb_admissions'].min():.0f} | Max : {df['nb_admissions'].max():.0f}")

In [ ]:
# Visualisation des données
fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(df['date'], df['nb_admissions'], alpha=0.7, linewidth=0.8)
ax.axhline(df['nb_admissions'].mean(), color='red', linestyle='--', label=f"Moyenne ({df['nb_admissions'].mean():.0f})")
ax.set_xlabel('Date')
ax.set_ylabel('Admissions quotidiennes')
ax.set_title('Admissions Hospitalières Quotidiennes (2020-2025)')
ax.legend()
plt.tight_layout()
plt.show()

## 2. Explication des Métriques d'Évaluation

### 2.1 MAE (Mean Absolute Error) - Erreur Absolue Moyenne

$$MAE = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|$$

**Interprétation** : Erreur moyenne en valeur absolue. Une MAE de 6 signifie que le modèle se trompe de ±6 admissions en moyenne.

### 2.2 RMSE (Root Mean Square Error)

$$RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}$$

**Interprétation** : Pénalise davantage les grosses erreurs. Si RMSE >> MAE, il y a des erreurs importantes ponctuelles.

### 2.3 MAPE (Mean Absolute Percentage Error)

$$MAPE = \frac{100}{n} \sum_{i=1}^{n} \left|\frac{y_i - \hat{y}_i}{y_i}\right|$$

**Interprétation** : Erreur en pourcentage. Une MAPE de 1.25% signifie que le modèle se trompe de 1.25% en moyenne.

### 2.4 R² (Coefficient de Détermination)

$$R^2 = 1 - \frac{\sum (y_i - \hat{y}_i)^2}{\sum (y_i - \bar{y})^2}$$

**Interprétation** : Proportion de variance expliquée. R² = 0.98 signifie que le modèle explique 98% de la variabilité.

In [ ]:
# Illustration des métriques
np.random.seed(42)
y_true = np.array([100, 150, 200, 180, 220])
y_pred = np.array([95, 160, 190, 185, 210])

mae = np.mean(np.abs(y_true - y_pred))
rmse = np.sqrt(np.mean((y_true - y_pred)**2))
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
r2 = 1 - np.sum((y_true - y_pred)**2) / np.sum((y_true - np.mean(y_true))**2)

print("Exemple simple avec 5 observations :")
print(f"Valeurs réelles : {y_true}")
print(f"Prédictions     : {y_pred}")
print(f"Erreurs         : {y_true - y_pred}")
print()
print(f"MAE  = {mae:.2f} (erreur moyenne de ±{mae:.0f})")
print(f"RMSE = {rmse:.2f}")
print(f"MAPE = {mape:.2f}% (erreur de {mape:.1f}% en moyenne)")
print(f"R²   = {r2:.4f} (explique {r2*100:.1f}% de la variance)")

## 3. Résultats de la Comparaison des Modèles

In [ ]:
# Résultats des tests (issus de nos expérimentations)
resultats = {
    'Modèle': ['Gradient Boosting', 'Random Forest', 'Ensemble', 'Prophet', 'ARIMA'],
    'MAE': [6.52, 7.84, 7.37, 54.53, 61.68],
    'RMSE': [9.89, 11.12, 10.41, 63.68, 76.91],
    'MAPE (%)': [1.25, 1.51, 1.45, 11.34, 13.44],
    'R²': [0.9842, 0.9800, 0.9825, 0.3435, 0.0424]
}

df_results = pd.DataFrame(resultats)
df_results = df_results.set_index('Modèle')

# Affichage avec style
def highlight_best(s):
    if s.name in ['MAE', 'RMSE', 'MAPE (%)']:
        is_best = s == s.min()
    else:  # R²
        is_best = s == s.max()
    return ['background-color: #90EE90' if v else '' for v in is_best]

df_results.style.apply(highlight_best)

In [ ]:
# Visualisation comparative
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

colors = ['#2ecc71', '#3498db', '#9b59b6', '#f39c12', '#e74c3c']
modeles = df_results.index.tolist()

# MAE
axes[0, 0].barh(modeles, df_results['MAE'], color=colors)
axes[0, 0].set_xlabel('MAE (admissions)')
axes[0, 0].set_title('MAE - Erreur Absolue Moyenne\n(plus bas = meilleur)')
for i, v in enumerate(df_results['MAE']):
    axes[0, 0].text(v + 1, i, f'{v:.2f}', va='center')

# RMSE
axes[0, 1].barh(modeles, df_results['RMSE'], color=colors)
axes[0, 1].set_xlabel('RMSE')
axes[0, 1].set_title('RMSE - Racine Erreur Quadratique Moyenne\n(plus bas = meilleur)')
for i, v in enumerate(df_results['RMSE']):
    axes[0, 1].text(v + 1, i, f'{v:.2f}', va='center')

# MAPE
axes[1, 0].barh(modeles, df_results['MAPE (%)'], color=colors)
axes[1, 0].set_xlabel('MAPE (%)')
axes[1, 0].set_title('MAPE - Erreur en Pourcentage\n(plus bas = meilleur)')
for i, v in enumerate(df_results['MAPE (%)']):
    axes[1, 0].text(v + 0.3, i, f'{v:.2f}%', va='center')

# R²
axes[1, 1].barh(modeles, df_results['R²'], color=colors)
axes[1, 1].set_xlabel('R²')
axes[1, 1].set_title('R² - Coefficient de Détermination\n(plus haut = meilleur)')
axes[1, 1].set_xlim(0, 1.1)
for i, v in enumerate(df_results['R²']):
    axes[1, 1].text(v + 0.02, i, f'{v:.4f}', va='center')

plt.tight_layout()
plt.savefig('../docs/comparaison_modeles.png', dpi=150, bbox_inches='tight')
plt.show()

print("📊 Graphique sauvegardé dans docs/comparaison_modeles.png")

## 4. Analyse des Résultats

### 🏆 Classement des modèles

1. **Gradient Boosting** - Meilleur sur toutes les métriques
2. **Ensemble** - Très proche, combine les forces des modèles
3. **Random Forest** - Excellent également
4. **Prophet** - Performance moyenne
5. **ARIMA** - Le moins performant

In [ ]:
# Radar chart pour visualiser les forces de chaque modèle
from math import pi

# Normaliser les métriques (0-1, inversé pour MAE/RMSE/MAPE)
df_norm = df_results.copy()
df_norm['MAE'] = 1 - (df_results['MAE'] - df_results['MAE'].min()) / (df_results['MAE'].max() - df_results['MAE'].min())
df_norm['RMSE'] = 1 - (df_results['RMSE'] - df_results['RMSE'].min()) / (df_results['RMSE'].max() - df_results['RMSE'].min())
df_norm['MAPE (%)'] = 1 - (df_results['MAPE (%)'] - df_results['MAPE (%)'].min()) / (df_results['MAPE (%)'].max() - df_results['MAPE (%)'].min())
# R² est déjà dans le bon sens

# Radar chart
categories = list(df_norm.columns)
N = len(categories)

angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))

for i, model in enumerate(df_norm.index[:3]):  # Top 3 modèles
    values = df_norm.loc[model].values.flatten().tolist()
    values += values[:1]
    ax.plot(angles, values, 'o-', linewidth=2, label=model, color=colors[i])
    ax.fill(angles, values, alpha=0.1, color=colors[i])

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)
ax.set_ylim(0, 1.1)
ax.set_title('Comparaison des 3 Meilleurs Modèles\n(plus grand = meilleur)', size=14, y=1.1)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))

plt.tight_layout()
plt.show()

## 5. Feature Engineering - Ce qui fait la différence

In [ ]:
# Démonstration de l'importance du feature engineering
print("📊 Nombre de features par approche :")
print()
print("ARIMA / Prophet (modèles statistiques) :")
print("   → Utilisent uniquement la série temporelle brute")
print("   → Pas de features supplémentaires")
print()
print("Random Forest / Gradient Boosting (version basique) :")
print("   → ~10 features : jour semaine, mois, lag_1, lag_7, ma_7...")
print("   → MAE ≈ 31")
print()
print("Random Forest / Gradient Boosting (version optimisée) :")
print("   → 135 features !")
print("   → MAE ≈ 6.5")
print()
print("➡️ Amélioration de -79% grâce au feature engineering")

In [ ]:
# Liste des catégories de features
features_categories = {
    'Temporelles de base': [
        'day_of_week', 'day_of_month', 'month', 'quarter', 'week_of_year', 'year'
    ],
    'Cycliques (sin/cos)': [
        'sin_day_week', 'cos_day_week', 'sin_month', 'cos_month', 'sin_day_month', 'cos_day_month'
    ],
    'Binaires': [
        'is_weekend', 'is_monday', 'is_friday', 'is_winter', 'is_spring', 'is_summer', 'is_autumn',
        'is_month_start', 'is_month_end'
    ],
    'Lags (historique)': [
        'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'lag_14', 'lag_21', 'lag_28', 'lag_30'
    ],
    'Moyennes mobiles': [
        'ma_3', 'ma_5', 'ma_7', 'ma_14', 'ma_21', 'ma_30'
    ],
    'Écarts-types': [
        'std_3', 'std_5', 'std_7', 'std_14', 'std_21', 'std_30'
    ],
    'Min/Max glissants': [
        'min_3', 'max_3', 'min_7', 'max_7', 'min_14', 'max_14', 'min_30', 'max_30', 'etc.'
    ],
    'EMA (exponentielle)': [
        'ema_7', 'ema_14', 'ema_30'
    ],
    'Tendances': [
        'trend_1d', 'trend_7d', 'trend_14d', 'trend_30d', 'pct_change_1d', 'pct_change_7d'
    ],
    'Même jour semaine précédente': [
        'same_day_last_week', 'same_day_2weeks_ago', 'same_day_4weeks_ago'
    ],
    'Ratios': [
        'ratio_to_ma7', 'ratio_to_ma30'
    ]
}

print("📋 Catégories de Features (135 au total) :")
print("="*50)
for cat, feats in features_categories.items():
    print(f"\n{cat} ({len(feats)} features) :")
    print(f"   {', '.join(feats[:5])}{'...' if len(feats) > 5 else ''}")

## 6. Conclusion

### Résumé des résultats

| Aspect | Résultat |
|--------|----------|
| **Meilleur modèle** | Gradient Boosting |
| **MAE** | 6.52 admissions (erreur de ±6 patients/jour) |
| **R²** | 0.98 (explique 98% de la variance) |
| **MAPE** | 1.25% (erreur relative très faible) |

### Pourquoi les modèles ML surpassent ARIMA/Prophet ?

1. **Feature engineering** : 135 variables capturant différents aspects temporels
2. **Non-linéarité** : Les modèles à base d'arbres capturent des relations complexes
3. **Interactions** : GB/RF peuvent modéliser les interactions entre variables

### Recommandations

- Utiliser **Gradient Boosting** en production
- Réentraîner régulièrement (mensuel) avec les nouvelles données
- Surveiller les métriques pour détecter une dégradation des performances

In [ ]:
# Résumé final
print("="*60)
print("                    RÉSUMÉ FINAL")
print("="*60)
print()
print("🏆 Meilleur modèle : Gradient Boosting")
print()
print("📊 Performances :")
print(f"   • MAE  = 6.52 (erreur de ±6 admissions/jour)")
print(f"   • R²   = 0.98 (98% de variance expliquée)")
print(f"   • MAPE = 1.25% (erreur relative)")
print()
print("📈 Amélioration vs ARIMA (baseline) :")
print(f"   • MAE  : 61.68 → 6.52 (-89%)")
print(f"   • R²   : 0.04 → 0.98 (+2200%)")
print()
print("🔑 Facteur clé : Feature Engineering (135 variables)")
print("="*60)